In [21]:
import os
os.environ['KAGGLE_CONFIG_DIR']='.'

In [ ]:
!kaggle competitions download -c titanic

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 51.8MB/s]


In [ ]:
!unzip titanic.zip && rm titanic.zip

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [22]:
_df = pd.read_csv("train.csv")
_df.shape

(891, 12)

In [23]:
_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [125]:
cols = ['PassengerId', 'Cabin', 'Ticket','Name']
df = _df.drop(cols,axis=1)

In [ ]:
df['Sex'] = df['Sex'].map(lambda x: 0 if x=='male' else 1)
df['Embarked'] = df['Embarked'].map(lambda x: 1 if x=='C' else 2 if x=='Q' else 3)
df['Age'] = df['Age'].map(lambda x: 28 if np.isnan(x) else x)

In [127]:
df = df.dropna().drop_duplicates()
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,3
1,1,1,1,38.0,1,0,71.2833,1
2,1,3,1,26.0,0,0,7.9250,3
3,1,1,1,35.0,1,0,53.1000,3
4,0,3,0,35.0,0,0,8.0500,3
...,...,...,...,...,...,...,...,...
885,0,3,1,39.0,0,5,29.1250,2
887,1,1,1,19.0,0,0,30.0000,3
888,0,3,1,28.0,1,2,23.4500,3
889,1,1,0,26.0,0,0,30.0000,1


In [47]:
fig = px.histogram(df, x="Survived")
fig.update_layout(bargap= 0.5)
fig.show()

In [57]:
df['Survived'].value_counts()/len(df)

0    0.595506
1    0.404494
Name: Survived, dtype: float64

In [40]:
fig =px.box(df, y='Age',x='Survived', color= 'Sex',points='all', hover_data = ['Pclass'])
fig.show()

In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


In [61]:
cv = StratifiedKFold(n_splits=5, shuffle=True)

In [63]:
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [103]:
train_features = ['Pclass', 'Sex', 'Age']
output_feature = 'Survived'

In [108]:
classification = LogisticRegression(max_iter=2000, class_weight='balanced', multi_class='ovr')

In [128]:
classification.fit(df[train_features], df[output_feature])

LogisticRegression(class_weight='balanced', max_iter=2000, multi_class='ovr')

In [129]:
y_predict = classification.predict(df[train_features])

In [130]:
accuracy_score(df[output_feature], y_predict)

0.7716129032258064

In [ ]:
scores=cross_val_score(classification,X_train,y_train,scoring='accuracy',cv=cv)
scores


In [118]:
test = pd.read_csv('test.csv')
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [119]:
test['Sex'] = test['Sex'].map(lambda x: 0 if x=='male' else 1)
test['Embarked'] = test['Embarked'].map(lambda x: 1 if x=='C' else 2 if x=='Q' else 3)
test['Age'] = test['Age'].map(lambda x: 28 if np.isnan(x) else x)

In [132]:
test['Survived']=classification.predict(test[train_features])

In [ ]:
test.isnull().sum()

In [19]:
x = sns.load_dataset('titanic')
x

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [121]:
def write_to_csv(dataframe, cols, filename='output.csv'):
  x = pd.DataFrame()
  for i in cols:
    x[i] = dataframe[i]
  x.to_csv(filename,index=False)

In [133]:
write_to_csv(test, ['PassengerId','Survived'])
!kaggle competitions submit -c titanic -f output.csv -m "Logistic Regression is used"

100% 2.77k/2.77k [00:00<00:00, 10.8kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster